In [1]:
import sys
from pathlib import Path
import torch
import pandas as pd

# Add project root to path
notebook_dir = Path().resolve()
project_root = notebook_dir.parent

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))


In [2]:
def count_parameters(model: torch.nn.Module, trainable_only: bool = False) -> dict:
    total_params = 0
    trainable_params = 0
    
    for param in model.parameters():
        num_params = param.numel()
        total_params += num_params
        if param.requires_grad:
            trainable_params += num_params
    
    frozen_params = total_params - trainable_params
    
    total_size_mb = (total_params * 4) / (1024 ** 2)
    trainable_size_mb = (trainable_params * 4) / (1024 ** 2)
    
    result = {
        'total': total_params,
        'trainable': trainable_params,
        'frozen': frozen_params,
        'total_size_mb': total_size_mb,
        'trainable_size_mb': trainable_size_mb,
    }
    
    return result


def print_model_summary(model: torch.nn.Module, model_name: str = "Model"):
    params = count_parameters(model)
    
    print(f"\n{'=' * 60}")
    print(f"{model_name} - Parameter Summary")
    print(f"{'=' * 60}")
    print(f"Total parameters:      {params['total']:,}")
    print(f"Trainable parameters:  {params['trainable']:,}")
    print(f"Frozen parameters:     {params['frozen']:,}")
    print(f"\nEstimated model size:")
    print(f"  Total:               {params['total_size_mb']:.2f} MB")
    print(f"  Trainable:           {params['trainable_size_mb']:.2f} MB")
    
    if params['total'] > 0:
        trainable_pct = (params['trainable'] / params['total']) * 100
        print(f"\nTrainable: {trainable_pct:.1f}% of total parameters")
    
    print(f"{'=' * 60}\n")
    
    return params


In [3]:
from src.models.model_loader import ModelLoader

loader = ModelLoader(project_root=project_root)

# All model configs that have checkpoints
model_configs = {
    'UNet 2C': ('unet_2', 'cnn'),
    'UNet 3C': ('unet_3', 'cnn'),
    'UNet++ 2C': ('unet++_2', 'cnn'),
    'UNet++ 3C': ('unet++_3', 'cnn'),
    'DeepLabV3 2C': ('deeplabv3_2', 'cnn'),
    'DeepLabV3 3C': ('deeplabv3_3', 'cnn'),
    'Phikon 2C': ('phikon_2', 'cnn'),
}

loader.load_all_models(model_configs)

/opt/anaconda3/envs/research_project/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded CNN: UNet 2C
Loaded CNN: UNet 3C
Loaded CNN: UNet++ 2C
Loaded CNN: UNet++ 3C
Loaded CNN: DeepLabV3 2C
Loaded CNN: DeepLabV3 3C


/opt/anaconda3/envs/research_project/lib/python3.12/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/opt/anaconda3/envs/research_project/lib/python3.12/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/anaconda3/envs/research_project/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


Loaded CNN: Phikon 2C
Loaded 7 models total


In [4]:
results = []

for model_name, model_info in loader.models.items():
    if model_info['type'] == 'cnn':
        model = model_info['model']
        params = count_parameters(model)
        
        results.append({
            'Model Name': model_name,
            'Config': model_info['config'].model.name,
            'Total Parameters': params['total'],
            'Trainable Parameters': params['trainable'],
            'Frozen Parameters': params['frozen'],
            'Total Size (MB)': params['total_size_mb'],
            'Trainable Size (MB)': params['trainable_size_mb'],
            'Trainable %': f"{(params['trainable'] / params['total'] * 100):.1f}%" if params['total'] > 0 else "0%"
        })
        
        print_model_summary(model, model_name=model_name)



UNet 2C - Parameter Summary
Total parameters:      24,436,514
Trainable parameters:  24,436,514
Frozen parameters:     0

Estimated model size:
  Total:               93.22 MB
  Trainable:           93.22 MB

Trainable: 100.0% of total parameters


UNet 3C - Parameter Summary
Total parameters:      24,436,659
Trainable parameters:  24,436,659
Frozen parameters:     0

Estimated model size:
  Total:               93.22 MB
  Trainable:           93.22 MB

Trainable: 100.0% of total parameters


UNet++ 2C - Parameter Summary
Total parameters:      26,078,754
Trainable parameters:  26,078,754
Frozen parameters:     0

Estimated model size:
  Total:               99.48 MB
  Trainable:           99.48 MB

Trainable: 100.0% of total parameters


UNet++ 3C - Parameter Summary
Total parameters:      26,078,899
Trainable parameters:  26,078,899
Frozen parameters:     0

Estimated model size:
  Total:               99.48 MB
  Trainable:           99.48 MB

Trainable: 100.0% of total parameters



In [ ]:
df = pd.DataFrame(results)

df = df.sort_values('Total Parameters')

print(df.to_string(index=False))

display(df)



MODEL PARAMETER SUMMARY
  Model Name                         Config  Total Parameters  Trainable Parameters  Frozen Parameters  Total Size (MB)  Trainable Size (MB) Trainable %
     UNet 2C      unet_resnet34_imagenet_2c          24436514              24436514                  0        93.217903            93.217903      100.0%
     UNet 3C      unet_resnet34_imagenet_3c          24436659              24436659                  0        93.218456            93.218456      100.0%
DeepLabV3 2C deeplabv3_resnet34_imagenet_2c          26007362              26007362                  0        99.210213            99.210213      100.0%
DeepLabV3 3C deeplabv3_resnet34_imagenet_3c          26007619              26007619                  0        99.211193            99.211193      100.0%
   UNet++ 2C    unet++_resnet34_imagenet_2c          26078754              26078754                  0        99.482552            99.482552      100.0%
   UNet++ 3C    unet++_resnet34_imagenet_3c          2607

,Model Name,Config,Total Parameters,Trainable Parameters,Frozen Parameters,Total Size (MB),Trainable Size (MB),Trainable %
0,UNet 2C,unet_resnet34_imagenet_2c,24436514,24436514,0,93.217903,93.217903,100.0%
1,UNet 3C,unet_resnet34_imagenet_3c,24436659,24436659,0,93.218456,93.218456,100.0%
4,DeepLabV3 2C,deeplabv3_resnet34_imagenet_2c,26007362,26007362,0,99.210213,99.210213,100.0%
5,DeepLabV3 3C,deeplabv3_resnet34_imagenet_3c,26007619,26007619,0,99.211193,99.211193,100.0%
2,UNet++ 2C,unet++_resnet34_imagenet_2c,26078754,26078754,0,99.482552,99.482552,100.0%
3,UNet++ 3C,unet++_resnet34_imagenet_3c,26078899,26078899,0,99.483105,99.483105,100.0%
6,Phikon 2C,phikon_v2_frozen_2c,309276418,5924610,303351808,1179.795906,22.600594,1.9%
